# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

In [1]:
import spacy
import nltk

#### Import the necessary libraries

In [2]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pycrfsuite


### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [3]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [4]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [5]:
def tuples_maker(inp, out):
   
    sample_data=[]
    for i in range(0,len(inp)):
        data=[]
        
        for y in range(0,len(inp[i])):
            tup=(inp[i][y],out[i][y])
            data.append(tup)
        sample_data.append(data)
    
    return sample_data

In [7]:
tuples=tuples_maker(input_sigs, output_labels)
tuples

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

In [8]:
len(tuples)

56

In [10]:
'''data=[]
t1=tuple((nltk.pos_tag([tuples[0][0][0]])))
((e1,e2),)=t1
t=e1,e2,tuples[0][0][1]
data.append(t)
data
'''

[('for', 'IN', 'FOR')]

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [11]:
def triples_maker(whole_data):

    sample_data=[]
    
    
    for i in range(0,len(whole_data)):
        triple=[]
        for y in range(0,len(whole_data[i])):
            
            t1=tuple((nltk.pos_tag([tuples[i][y][0]])))
            ((e1,e2),)=t1
            data=(e1,e2,tuples[i][y][1])
            triple.append(data)
        sample_data.append(triple)
    return sample_data

In [12]:
sample_data = triples_maker(tuples)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'NN', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'NN', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'NN', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [13]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [14]:
from sklearn.model_selection import train_test_split

def get_features(doc):
    return [token_to_features(doc,i) for i in range(len(doc))]

def get_labels(doc):
    return [label for (token,pos,label) in doc]

X=[get_features(doc) for doc in sample_data]

y=[get_labels(doc) for doc in sample_data]



In [15]:
X,y

([[['bias',
    'word.lower=for',
    'word[-3:]=for',
    'word[-2:]=or',
    'word.isupper=False',
    'word.istitle=False',
    'word.isdigit=False',
    'postag=IN',
    'BOS',
    '+1:word.lower=5',
    '+1:word.istitle=False',
    '+1:word.isupper=False',
    '+1:word.isdigit=True',
    '+1:postag=CD'],
   ['bias',
    'word.lower=5',
    'word[-3:]=5',
    'word[-2:]=5',
    'word.isupper=False',
    'word.istitle=False',
    'word.isdigit=True',
    'postag=CD',
    '-1:word.lower=for',
    '-1:word.istitle=False',
    '-1:word.isupper=False',
    '-1:word.isdigit=False',
    '-1:postag=IN',
    '+1:word.lower=to',
    '+1:word.istitle=False',
    '+1:word.isupper=False',
    '+1:word.isdigit=False',
    '+1:postag=TO'],
   ['bias',
    'word.lower=to',
    'word[-3:]=to',
    'word[-2:]=to',
    'word.isupper=False',
    'word.istitle=False',
    'word.isdigit=False',
    'postag=TO',
    '-1:word.lower=5',
    '-1:word.istitle=False',
    '-1:word.isupper=False',
    '-1:word

In [16]:
X_train,X_test,y_train,y_test= train_test_split(X, y)


### Training the CRF model with the features extracted using the feature extractor method

In [17]:
trainer=pycrfsuite.Trainer()

# Submit training data to the trainer

X,y

for xseq,yseq in zip(X,y):
    trainer.append(xseq,yseq)

# Set the parameters of the model
trainer.set_params({
    'max_iterations' : 1000, 
    'c1' : 0.1, 
    'c2' : 0.01
})


# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('prescription_parser.model')


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1006
Seconds required: 0.008

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 733.220378
Feature norm: 1.000000
Error norm: 148.884949
Active features: 995
Line search trials: 1
Line search step: 0.005177
Seconds required for this iteration: 0.001

***** Iteration #2 *****
Loss: 458.668348
Feature norm: 5.711035
Error norm: 183.358308
Active features: 982
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #3 *****
Loss: 332.411801
Feature norm: 7.019698
Error norm: 85.820971
Active features: 976
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.

### Predicting the test data with the built model

In [18]:
tagger= pycrfsuite.Tagger()
tagger.open('prescription_parser.model')

predicted_tags = tagger.tag(y_test[5])

In [19]:
predicted_tags

['Method',
 'Qty',
 'Form',
 'Frequency',
 'FOR',
 'Duration',
 'DurationUnit',
 'FOR',
 'Duration',
 'DurationUnit']

In [45]:
sig1= "5 days 2 tabs at bed"

In [48]:
from nltk.tokenize import word_tokenize
test=nltk.pos_tag(word_tokenize(sig1))

In [50]:
X=[token_to_features(test,i) for i in range(len(test))]

In [51]:
predictions= tagger.tag(X)

In [52]:
predictions

['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']

### Putting all the prediction logic inside a predict method

In [53]:
def predict(sig):

    
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    test=nltk.pos_tag(word_tokenize(sig))
    X=[token_to_features(test,i) for i in range(len(test))]
    predictions= tagger.tag(X)
    print(sig)
    print([predictions])
    
    return predictions

### Sample predictions

In [54]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [55]:
predictions = predict("2 capsu for 10 day at bed")

2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'PeriodUnit', 'AT', 'WHEN']]


In [56]:
predictions = predict("2 capsu for 10 days at bed")

2 capsu for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [57]:
predictions = predict("5 days 2 tabs at bed")

5 days 2 tabs at bed
[['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']]


In [58]:
predictions = predict("3 tabs qid x 10 weeks")

3 tabs qid x 10 weeks
[['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [59]:
predictions = predict("x 30 days")

x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [60]:
predictions = predict("x 20 months")

x 20 months
[['FOR', 'Duration', 'DurationUnit']]


In [61]:
predictions = predict("take 2 tabs po tid for 10 days")

take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']]


In [62]:
predictions = predict("take 2 capsules po every 6 hours")

take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']]


In [63]:
predictions = predict("inject 2 units pu tid")

inject 2 units pu tid
[['Method', 'Qty', 'Form', 'Frequency', 'TID']]


In [64]:
predictions = predict("swallow 3 caps tid by mouth")

swallow 3 caps tid by mouth
[['Method', 'Qty', 'Form', 'TID', 'BY', 'PO']]


In [65]:
predictions = predict("inject 3 units orally")

inject 3 units orally
[['Method', 'Qty', 'Form', 'PO']]


In [66]:
predictions = predict("orally take 3 tabs tid")

orally take 3 tabs tid
[['PO', 'Method', 'Qty', 'Form', 'TID']]


In [67]:
predictions = predict("by mouth take three caps")

by mouth take three caps
[['BY', 'PO', 'Method', 'Qty', 'Form']]


In [68]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [69]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [70]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'BID', 'PO', 'AT', 'WHEN']]


In [71]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']]


In [72]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [73]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
[['Method', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Period', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit', 'AND', 'EVERY', 'Period', 'PeriodUnit']]


In [74]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
[['FOR', 'Duration', 'Qty', 'Form']]


In [75]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'WITH', 'FOOD']]


In [76]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [77]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [78]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
[['Method', 'Qty', 'Form']]


In [79]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [80]:
prediction = predict("take two to four tabs")

take two to four tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [81]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']]


In [82]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [83]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [84]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [85]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [86]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [87]:
prediction = predict("every 60 mins")

every 60 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [88]:
prediction = predict("every 10 mins")

every 10 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [89]:
prediction = predict("every two to four months")

every two to four months
[['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [90]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [91]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
[['EVERY', 'Period', 'TO', 'DurationMax', 'DurationUnit', 'Method', 'Qty', 'Form']]


In [92]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
[['Frequency', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form']]


In [93]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit']]


In [94]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
[['PO', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]


In [95]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [96]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [97]:
prediction = predict("10 tabs")

10 tabs
[['Qty', 'Form']]


In [99]:
from sklearn.metrics import classification_report

all_true, all_pred = [], []

for i in range(len(X_test)):
    all_true.extend(y_test[i])
    all_pred.extend(tagger.tag(X_test[i]))

print(classification_report(all_true, all_pred))

              precision    recall  f1-score   support

         AND       1.00      1.00      1.00         1
          AT       1.00      1.00      1.00         1
      BEFORE       1.00      1.00      1.00         4
          BY       1.00      1.00      1.00         1
    Duration       1.00      1.00      1.00         3
 DurationMax       0.00      0.00      0.00         1
DurationUnit       1.00      0.67      0.80         3
       EVERY       1.00      1.00      1.00         3
        FOOD       1.00      1.00      1.00         1
         FOR       1.00      1.00      1.00         3
        Form       1.00      1.00      1.00         5
   Frequency       0.00      0.00      0.00         0
           M       1.00      1.00      1.00         1
      Method       1.00      1.00      1.00         4
          PO       1.00      1.00      1.00         1
      Period       1.00      1.00      1.00         4
   PeriodMax       0.50      1.00      0.67         1
  PeriodUnit       0.83    

/Users/harshwardhanbabel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshwardhanbabel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
